In [1]:
# Import the libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("/var/basefolder_smit/llama-2-13b")
tokenizer = AutoTokenizer.from_pretrained("/var/basefolder_smit/llama-2-13b")

# Load the dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# Define the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("output")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/anaconda3/envs/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/root/anaconda3/envs/venv/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /root/anaconda3/envs/venv did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


[2023-07-26 13:19:55,513] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


OSError: Can't load the configuration of '/var/basefolder_smit/llama-2-13b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/var/basefolder_smit/llama-2-13b' is the correct path to a directory containing a config.json file